# CS 181, Spring 2016
## Homework 1, Problem 3

### 1. Prepare
#### Import

In [1]:
# import packages
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

#### Read in data

In [3]:
# read in data:
# (1) store data name
csv_filename = 'year-sunspots-republicans.csv'
# (2) creat column names
years  = []
republican_counts = []
sunspot_counts = []

with open(csv_filename, 'r') as csv_fh:
    
    # Parse as a CSV file.
    reader = csv.reader(csv_fh)
    
    # Skip the header line.
    next(reader, None)
    
    # Loop over the file.
    for row in reader:
        
        # Store the data.
        years.append(float(row[0]))
        sunspot_counts.append(float(row[1]))
        republican_counts.append(float(row[2]))
        
# Turn the data into numpy arrays.
years  = np.array(years)
republican_counts = np.array(republican_counts)
sunspot_counts = np.array(sunspot_counts)
last_year = 1985

# select data before 1985
sunspot = sunspot_counts[years<last_year]
republican = republican_counts[years<last_year]

#### plot data

In [5]:
# Plot the data.
fig, ax = plt.subplots(3, 1,figsize=(15, 18))
plt.suptitle("Plot of Data", fontsize=25, y = 0.92)
ax[0].scatter(years, republican_counts, s=70)
ax[0].set_xlabel("Year")
ax[0].set_ylabel("Number of Republicans in Congress")

ax[1].scatter(years, sunspot_counts, s=70)
ax[1].set_xlabel("Year")
ax[1].set_ylabel("Number of Sunspots")

ax[2].scatter(sunspot, republican, s=70)
ax[2].set_xlabel("Number of Sunspots")
ax[2].set_ylabel("Number of Republicans in Congress")

plt.savefig('data.png')

### A. years vs number of Republicans
#### simplest version

In [4]:
###################################################
#          years vs number of Republicans         #
###################################################
#(1) the simplest basis: 
# matrix for input
X = np.vstack((np.ones(years.shape), years)).T
##  1st column: offset(1.0)
##  2nd column: time

# Nothing fancy for outputs.
Y = republican_counts

# Find the regression weights using the Moore-Penrose pseudoinverse.
w = np.linalg.solve(np.dot(X.T, X) , np.dot(X.T, Y))

# compute mean square error
Yhat  = np.dot(X, w)
mse = mean_squared_error(republican_counts, Yhat)

# Compute the regression line on a grid of inputs.
grid_years = np.linspace(1960, 2005, 200)
grid_X = np.vstack((np.ones(grid_years.shape), grid_years))
grid_Yhat  = np.dot(grid_X.T, w)

# TODO: plot and report sum of squared error for each basis
# Plot the data and the regression line.
plt.figure(figsize=(15,6))
plt.plot(years, republican_counts, 'o', grid_years, grid_Yhat, '-')
plt.xlabel("Year")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse, ha="center")
plt.savefig('simplest.png')

#### with basis functions 

In [5]:
#(2) basis functions:
def make_basis_a(i):
    X = np.ones(i.shape).T   
    for j in range(1, 6):
        X = np.vstack((X, i**j))
    return X.T

def make_basis_b(i):
    X = np.ones(i.shape).T   
    for j in range(1960, 2015, 5):
        X = np.vstack((X, np.exp(np.divide(np.subtract(0,(i-j)**2),25))))
    return X.T

def make_basis_c(i):
    X = np.ones(i.shape).T   
    for j in range(1, 6):
        X = np.vstack((X, np.cos(i/j)))
    return X.T

def make_basis_d(i):
    X = np.ones(i.shape).T   
    for j in range(1, 26):
        X = np.vstack((X, np.cos(i/j)))
    return X.T

In [6]:
# matrix for inputs
X_a = make_basis_a(years)
X_b = make_basis_b(years)
X_c = make_basis_c(years)
X_d = make_basis_d(years)

# Nothing fancy for outputs.
Y = republican_counts

# Find the regression weights using the Moore-Penrose pseudoinverse.
w_a = np.linalg.solve(np.dot(X_a.T, X_a) , np.dot(X_a.T, Y))
w_b = np.linalg.solve(np.dot(X_b.T, X_b) , np.dot(X_b.T, Y))
w_c = np.linalg.solve(np.dot(X_c.T, X_c) , np.dot(X_c.T, Y))
w_d = np.linalg.solve(np.dot(X_d.T, X_d) , np.dot(X_d.T, Y))

# compute mean square error
Yhat_a  = np.dot(X_a, w_a)
mse_a = mean_squared_error(republican_counts, Yhat_a)

Yhat_b  = np.dot(X_b, w_b)
mse_b = mean_squared_error(republican_counts, Yhat_b)

Yhat_c  = np.dot(X_c, w_c)
mse_c = mean_squared_error(republican_counts, Yhat_c)

Yhat_d  = np.dot(X_d, w_d)
mse_d = mean_squared_error(republican_counts, Yhat_d)

# Compute the regression line on a grid of inputs.
grid_years = np.linspace(1960, 2005, 200)
grid_X_a = make_basis_a(grid_years)
grid_Yhat_a  = np.dot(grid_X_a, w_a)

grid_years = np.linspace(1960, 2005, 200)
grid_X_b = make_basis_b(grid_years)
grid_Yhat_b  = np.dot(grid_X_b, w_b)

grid_years = np.linspace(1960, 2005, 200)
grid_X_c = make_basis_c(grid_years)
grid_Yhat_c  = np.dot(grid_X_c, w_c)

grid_years = np.linspace(1960, 2005, 200)
grid_X_d = make_basis_d(grid_years)
grid_Yhat_d  = np.dot(grid_X_d, w_d)

# TODO: plot and report sum of squared error for each basis
# Plot the data and the regression line.
plt.figure(figsize=(15,6))
plt.plot(years, republican_counts, 'o', grid_years, grid_Yhat_a, '-')
plt.xlabel("Year")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse_a, ha="center")
plt.savefig('a.png')

plt.figure(figsize=(15,6))
plt.plot(years, republican_counts, 'o', grid_years, grid_Yhat_b, '-')
plt.xlabel("Year")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse_b, ha="center")
plt.savefig('b.png')

plt.figure(figsize=(15,6))
plt.plot(years, republican_counts, 'o', grid_years, grid_Yhat_c, '-')
plt.xlabel("Year")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse_a, ha="center")
plt.savefig('c.png')

plt.figure(figsize=(15,6))
plt.plot(years, republican_counts, 'o', grid_years, grid_Yhat_d, '-')
plt.xlabel("Year")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse_d, ha="center")
plt.savefig('d.png')


### B. sunspots vs number of republicans 
#### simplest version

In [12]:
###################################################
#       sunspots vs number of republicans         #
###################################################
#(1) the simplest basis: 
# matrix for input
X_0_2 = np.vstack((np.ones(sunspot.shape), sunspot)).T
##  1st column: time
##  2nd column: offset(1.0)

# Nothing fancy for outputs.
Y_2 = republican

# Find the regression weights using the Moore-Penrose pseudoinverse.
w_2 = np.linalg.solve(np.dot(X_0_2.T, X_0_2) , np.dot(X_0_2.T, Y_2))

# compute mean square error
Yhat_2  = np.dot(X_0_2, w_2)
mse_2 = mean_squared_error(republican, Yhat_2)

# Compute the regression line on a grid of inputs.
grid_sunspot = np.linspace(10.2, 159, 200)
grid_X_0_2 = np.vstack((np.ones(grid_sunspot.shape), grid_sunspot))
grid_Yhat_2  = np.dot(grid_X_0_2.T, w_2)

# TODO: plot and report sum of squared error for each basis
# Plot the data and the regression line.
plt.figure(figsize=(15,6))
plt.plot(sunspot, republican, 'o', grid_sunspot, grid_Yhat_2, '-')
plt.xlabel("Number of Sunspots")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse, ha="center")
plt.savefig('simplest_2.png')

#### with basis functions 

In [13]:
# (2) matrix for inputs:
X_a_2 = make_basis_a(sunspot)
X_c_2 = make_basis_c(sunspot)
X_d_2 = make_basis_d(sunspot)

# Nothing fancy for outputs.
Y_2 = republican

# Find the regression weights using the Moore-Penrose pseudoinverse.
w_a_2 = np.linalg.solve(np.dot(X_a_2.T, X_a_2) , np.dot(X_a_2.T, Y_2))
w_c_2 = np.linalg.solve(np.dot(X_c_2.T, X_c_2) , np.dot(X_c_2.T, Y_2))
w_d_2 = np.linalg.solve(np.dot(X_d_2.T, X_d_2) , np.dot(X_d_2.T, Y_2))

# compute mean square error
Yhat_a_2  = np.dot(X_a_2, w_a_2)
mse_a_2 = mean_squared_error(Y_2, Yhat_a_2)

Yhat_c_2  = np.dot(X_c_2, w_c_2)
mse_c_2 = mean_squared_error(Y_2, Yhat_c_2)

Yhat_d_2  = np.dot(X_d_2, w_d_2)
mse_d_2 = mean_squared_error(Y_2, Yhat_d_2)

# Compute the regression line on a grid of inputs.
grid_sunspot = np.linspace(10.2, 159, 200)
grid_X_a_2 = make_basis_a(grid_sunspot)
grid_Yhat_a_2  = np.dot(grid_X_a_2, w_a_2)

grid_sunspot = np.linspace(10.2, 159, 200)
grid_X_c_2 = make_basis_c(grid_sunspot)
grid_Yhat_c_2  = np.dot(grid_X_c_2, w_c_2)

grid_sunspot = np.linspace(10.2, 159, 200)
grid_X_d_2 = make_basis_d(grid_sunspot)
grid_Yhat_d_2  = np.dot(grid_X_d_2, w_d_2)

# TODO: plot and report sum of squared error for each basis
# Plot the data and the regression line.
plt.figure(figsize=(15,6))
plt.plot(sunspot, republican, 'o', grid_sunspot, grid_Yhat_a_2, '-')
plt.xlabel("Number of Sunspots")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse_a_2, ha="center")
plt.savefig('a_2.png')

plt.figure(figsize=(15,6))
plt.plot(sunspot, republican, 'o', grid_sunspot, grid_Yhat_c_2, '-')
plt.xlabel("Number of Sunspots")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse_c_2, ha="center")
plt.savefig('c_2.png')

plt.figure(figsize=(15,6))
plt.plot(sunspot, republican, 'o', grid_sunspot, grid_Yhat_d_2, '-')
plt.xlabel("Number of Sunspots")
plt.ylabel("Number of Republicans in Congress")
plt.figtext(0.5, 0, "MSE = %s" % mse_d_2, ha="center")
plt.savefig('d_2.png')